<a href="https://colab.research.google.com/github/FaridRash/eurusd-regression-forecasting/blob/main/Farid/Farid_1st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Githhub

In [1]:
!git clone https://github.com/FaridRash/eurusd-regression-forecasting

Cloning into 'eurusd-regression-forecasting'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 65 (delta 24), reused 12 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 5.12 MiB | 8.24 MiB/s, done.
Resolving deltas: 100% (24/24), done.


#Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#Dataset

In [3]:
df = pd.read_csv('/content/eurusd-regression-forecasting/Dataset/EURUSD-2000-2020-15m.csv')
display(df.head())

,DATE_TIME,HIGH,LOW,OPEN,CLOSE
0,2000.01.03 00:00:00,1.0080,1.0073,1.0073,1.0077
1,2000.01.03 00:15:00,1.0087,1.0076,1.0078,1.0086
2,2000.01.03 00:30:00,1.0089,1.0079,1.0087,1.0079
3,2000.01.03 00:45:00,1.0132,1.0078,1.0078,1.0128
4,2000.01.03 01:00:00,1.0133,1.0120,1.0129,1.0122


In [4]:
df = df.drop('DATE_TIME', axis=1)
df = df.reset_index(drop=True)
display(df.head())

,HIGH,LOW,OPEN,CLOSE
0,1.0080,1.0073,1.0073,1.0077
1,1.0087,1.0076,1.0078,1.0086
2,1.0089,1.0079,1.0087,1.0079
3,1.0132,1.0078,1.0078,1.0128
4,1.0133,1.0120,1.0129,1.0122


#Preprocessing

##Shrinking

In [5]:
df_shrinked = df.tail(40000).reset_index(drop=True)
display(df_shrinked.head())
display(df_shrinked.tail())

,HIGH,LOW,OPEN,CLOSE
0,1.15593,1.15564,1.15578,1.15577
1,1.15603,1.15512,1.15580,1.15516
2,1.15553,1.15516,1.15518,1.15532
3,1.15532,1.15480,1.15529,1.15482
4,1.15553,1.15482,1.15483,1.15552


,HIGH,LOW,OPEN,CLOSE
39995,1.09007,1.08790,1.08803,1.08895
39996,1.08936,1.08807,1.08895,1.08872
39997,1.09118,1.08853,1.08872,1.09086
39998,1.09086,1.08946,1.09086,1.08970
39999,1.09060,1.08953,1.08974,1.08965


##Target Allocation

In [6]:
df_shrinked['target'] = (df_shrinked['CLOSE'].shift(-1) - df_shrinked['CLOSE']) * 10000
df_shrinked = df_shrinked.dropna()
display(df.head())
display(df.tail())

,HIGH,LOW,OPEN,CLOSE
0,1.0080,1.0073,1.0073,1.0077
1,1.0087,1.0076,1.0078,1.0086
2,1.0089,1.0079,1.0087,1.0079
3,1.0132,1.0078,1.0078,1.0128
4,1.0133,1.0120,1.0129,1.0122


,HIGH,LOW,OPEN,CLOSE
500746,1.09007,1.08790,1.08803,1.08895
500747,1.08936,1.08807,1.08895,1.08872
500748,1.09118,1.08853,1.08872,1.09086
500749,1.09086,1.08946,1.09086,1.08970
500750,1.09060,1.08953,1.08974,1.08965


##Moving Average

In [7]:
df_shrinked['MA_5'] = df_shrinked['CLOSE'].rolling(window=5).mean()
df_shrinked['MA_20'] = df_shrinked['CLOSE'].rolling(window=20).mean()
df_shrinked = df_shrinked.dropna()

In [8]:
display(df_shrinked.head())
display(df_shrinked.tail())

,HIGH,LOW,OPEN,CLOSE,target,MA_5,MA_20
19,1.15559,1.15519,1.15519,1.15558,-1.0,1.155284,1.155382
20,1.15565,1.15543,1.15556,1.15548,1.5,1.155344,1.155367
21,1.15568,1.15545,1.15549,1.15563,0.0,1.155448,1.155390
22,1.15568,1.15559,1.15563,1.15563,-0.3,1.155502,1.155406
23,1.15584,1.15558,1.15563,1.15560,0.6,1.155584,1.155445


,HIGH,LOW,OPEN,CLOSE,target,MA_5,MA_20
39994,1.09087,1.08771,1.08959,1.08802,9.3,1.089740,1.091733
39995,1.09007,1.08790,1.08803,1.08895,-2.3,1.089362,1.091597
39996,1.08936,1.08807,1.08895,1.08872,21.4,1.089314,1.091522
39997,1.09118,1.08853,1.08872,1.09086,-11.6,1.089228,1.091520
39998,1.09086,1.08946,1.09086,1.08970,-0.5,1.089250,1.091344


##Adding the history

In [9]:
# Adding the history
for col in ['HIGH', 'LOW', 'OPEN', 'CLOSE', 'MA_5', 'MA_20']:
    for i in range(1, 4):
        df_shrinked[f'{col}_prev_{i}'] = df_shrinked[col].shift(i)

df_shrinked = df_shrinked.dropna().reset_index(drop=True)

display(df_shrinked.head())

,HIGH,LOW,OPEN,CLOSE,target,MA_5,MA_20,HIGH_prev_1,HIGH_prev_2,HIGH_prev_3,...,OPEN_prev_3,CLOSE_prev_1,CLOSE_prev_2,CLOSE_prev_3,MA_5_prev_1,MA_5_prev_2,MA_5_prev_3,MA_20_prev_1,MA_20_prev_2,MA_20_prev_3
0,1.15568,1.15559,1.15563,1.15563,-0.3,1.155502,1.155406,1.15568,1.15565,1.15559,...,1.15519,1.15563,1.15548,1.15558,1.155448,1.155344,1.155284,1.155390,1.155367,1.155382
1,1.15584,1.15558,1.15563,1.15560,0.6,1.155584,1.155445,1.15568,1.15568,1.15565,...,1.15556,1.15563,1.15563,1.15548,1.155502,1.155448,1.155344,1.155406,1.155390,1.155367
2,1.15579,1.15553,1.15560,1.15566,0.3,1.155600,1.155452,1.15584,1.15568,1.15568,...,1.15549,1.15560,1.15563,1.15563,1.155584,1.155502,1.155448,1.155445,1.155406,1.155390
3,1.15569,1.15559,1.15567,1.15569,0.9,1.155642,1.155466,1.15579,1.15584,1.15568,...,1.15563,1.15566,1.15560,1.15563,1.155600,1.155584,1.155502,1.155452,1.155445,1.155406
4,1.15590,1.15569,1.15569,1.15578,2.7,1.155672,1.155473,1.15569,1.15579,1.15584,...,1.15563,1.15569,1.15566,1.15560,1.155642,1.155600,1.155584,1.155466,1.155452,1.155445


#Splitting

In [10]:
from sklearn.model_selection import train_test_split

X = df_shrinked[['HIGH', 'LOW', 'OPEN', 'CLOSE', 'MA_5', 'MA_20', 'HIGH_prev_1', 'LOW_prev_1', 'OPEN_prev_1', 'CLOSE_prev_1', 'MA_5_prev_1', 'MA_20_prev_1']]
y = df_shrinked['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (31981, 12)
X_test shape: (7996, 12)
y_train shape: (31981,)
y_test shape: (7996,)


#Standardization

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('X_train_scaled shape:', X_train_scaled.shape)
print('X_test_scaled shape:', X_test_scaled.shape)

X_train_scaled shape: (31981, 12)
X_test_scaled shape: (7996, 12)


#Modeling

##StackingRegressor

In [12]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Define base models
estimators = [
    ('lr', LinearRegression()),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(random_state=42))
]

# Create Stacking Regressor model
stacking_model = StackingRegressor(estimators=estimators)

# Train the model
stacking_model.fit(X_train_scaled, y_train)

print("Stacking Regressor model trained successfully.")

Stacking Regressor model trained successfully.


In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions on the test set
y_pred_stacked = stacking_model.predict(X_test_scaled)

# Calculate evaluation metrics
mae_stacked = mean_absolute_error(y_test, y_pred_stacked)
mse_stacked = mean_squared_error(y_test, y_pred_stacked)
r2_stacked = r2_score(y_test, y_pred_stacked)

print("Stacking Regressor Model Evaluation:")
print(f"Mean Absolute Error (MAE): {mae_stacked:.4f}")
print(f"Mean Squared Error (MSE): {mse_stacked:.4f}")
print(f"R-squared (R2): {r2_stacked:.4f}")

Stacking Regressor Model Evaluation:
Mean Absolute Error (MAE): 2.8246
Mean Squared Error (MSE): 20.5451
R-squared (R2): 0.0000


In [14]:
# Access the Random Forest Regressor from the stacking model
rf_from_stacking = stacking_model.named_estimators_['rf']

# Make predictions using the Random Forest Regressor
y_pred_rf_stacked = rf_from_stacking.predict(X_test_scaled)

# Calculate evaluation metrics for the Random Forest Regressor
mae_rf_stacked = mean_absolute_error(y_test, y_pred_rf_stacked)
mse_rf_stacked = mean_squared_error(y_test, y_pred_rf_stacked)
r2_rf_stacked = r2_score(y_test, y_pred_rf_stacked)

print("Random Forest Regressor (from Stacking Model) Evaluation:")
print(f"Mean Absolute Error (MAE): {mae_rf_stacked:.4f}")
print(f"Mean Squared Error (MSE): {mse_rf_stacked:.4f}")
print(f"R-squared (R2): {r2_rf_stacked:.4f}")

Random Forest Regressor (from Stacking Model) Evaluation:
Mean Absolute Error (MAE): 2.9830
Mean Squared Error (MSE): 21.9832
R-squared (R2): -0.0700


In [15]:
# Access the Linear Regressor from the stacking model
lr_from_stacking = stacking_model.named_estimators_['lr']

# Make predictions using the Random Forest Regressor
y_pred_lr_stacked = lr_from_stacking.predict(X_test_scaled)

# Calculate evaluation metrics for the Random Forest Regressor
mae_lr_stacked = mean_absolute_error(y_test, y_pred_lr_stacked)
mse_lr_stacked = mean_squared_error(y_test, y_pred_lr_stacked)
r2_lr_stacked = r2_score(y_test, y_pred_lr_stacked)

print("Linear Regressor (from Stacking Model) Evaluation:")
print(f"Mean Absolute Error (MAE): {mae_lr_stacked:.4f}")
print(f"Mean Squared Error (MSE): {mse_lr_stacked:.4f}")
print(f"R-squared (R2): {r2_lr_stacked:.4f}")

Linear Regressor (from Stacking Model) Evaluation:
Mean Absolute Error (MAE): 2.8246
Mean Squared Error (MSE): 20.5367
R-squared (R2): 0.0004


In [16]:
# Access the XGBoost from the stacking model
xgb_from_stacking = stacking_model.named_estimators_['xgb']

# Make predictions using the Random Forest Regressor
y_pred_xgb_stacked = xgb_from_stacking.predict(X_test_scaled)

# Calculate evaluation metrics for the Random Forest Regressor
mae_xgb_stacked = mean_absolute_error(y_test, y_pred_xgb_stacked)
mse_xgb_stacked = mean_squared_error(y_test, y_pred_xgb_stacked)
r2_xgb_stacked = r2_score(y_test, y_pred_xgb_stacked)

print("XGBoost (from Stacking Model) Evaluation:")
print(f"Mean Absolute Error (MAE): {mae_xgb_stacked:.4f}")
print(f"Mean Squared Error (MSE): {mse_xgb_stacked:.4f}")
print(f"R-squared (R2): {r2_xgb_stacked:.4f}")

XGBoost (from Stacking Model) Evaluation:
Mean Absolute Error (MAE): 2.8973
Mean Squared Error (MSE): 21.6248
R-squared (R2): -0.0525
